# Protein-protein interaction: competitive binding model

In this notebook, we use the model described in [Roehrl *et al*, Biochemistry 43.51 (2004): 16056-16066](https://pubs.acs.org/doi/full/10.1021/bi048233g?casa_token=rlvgtYAYetcAAAAA%3Av59DOeTp14qDL3OZDasWt-_LJYpS1buuMrQI9McanroG3nbRCansk8UERtaN8XlYoF2zyAInV6gE2Fqx) to fit experimental curves.


The notebook expects a certain format (`.csv` file):

| L_T (uM) | C1 replicate 1 (%) | ... | C2 replicate 1 (%) | ... |
|----------|:------------------:|:---:|:------------------:|:---:|
| 0.01     | 56.23              | ... | ...                | ... |
| 0.1      | ...                | ... | ...                | ... |
| 10       | ...                | ... | ...                | ... |


The values should be separated by a comma (`,`) and the decimals indicated by a dot (`.`). The names of the columns will be used for reporting.

The first column should be in `uM` units and the others in `%` (between 0 and 100). 

**Important**: The notebook assumes that there are two conditions in the `.csv` file, each with the same number of replicates.


## What the notebook does

In this notebook, we first load the data and normalize the values:
- The first column (`L_T`) is converted to M
- All the others columns (conditions) are divided by 100
- The smallest value of all conditions is subtracted from all conditions columns
- The conditions columns are split in condition1 and condition2

Then we fit the following:
- Individual replicates, save figures and summary results
- Fit all replicates from a condition together, save figure and summary


## How to use the notebook

Jupyter notebook are broken down in cells. Each cell can be run individually but they should be run in the correct order.

You can run a single cell (`shift`+`enter` or click on `run` in the toolbar) or the whole notebook (`Cell > Run all`).

The first cell imports all the necessary packages and defines some functions.


In [ ]:
# import packages:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas


def uM_to_M(c):
    """Convert uM to M"""
    return c / 1_000_000.0


def M_to_uM(c):
    """Convert M to uM"""
    return c * 1_000_000.0


def f_sb_model(l_t, kd_2, r_t, l_st, kd_1):
    """Model F_sb based on the other concentrations and interaction constants.

    All parameters are in M units.
    """
    d = kd_1 + kd_2 + l_st + l_t - r_t
    e = (l_t - r_t) * kd_1 + (l_st - r_t) * kd_2 + kd_1 * kd_2
    f = -kd_1 * kd_2 * r_t

    h = d * d - 3 * e

    # G
    g_num = -2.0 * pow(d, 3) + 9.0 * d * e - 27.0 * f
    g_den = 2.0 * np.sqrt(pow(h, 3))
    g_div = g_num / g_den

    G = np.arccos(g_div)

    # final value
    num = 2.0 * np.sqrt(h) * np.cos(G / 3.0) - d
    den = 3.0 * kd_1 + num

    return num / den


def get_fitting_func(l_st, kd_1):
    """Return a function that takes l_t data as input and can be used to fit kd_2 and r_t."""
    return lambda l_t, kd_2, r_t: f_sb_model(l_t, kd_2, r_t=r_t, l_st=l_st, kd_1=kd_1)


def fit_curve(x, y, kd_2_initial, r_t_initial, l_st, kd_1):
    """Fit the model to x,y data.

    All inputs are in M unit."""
    results = {}

    # fitting
    fitted_parameters, errors = curve_fit(
        get_fitting_func(l_st, kd_1),  # fitting function
        x,
        y,
        (kd_2_initial, r_t_initial),  # initial parameters
        bounds = (0, np.inf), # bounds
    )

    kd2_fitted, r_t_fitted = fitted_parameters

    print(
        f"Fitted values: kd_2={M_to_uM(kd2_fitted)} uM and r_t={M_to_uM(r_t_fitted)} uM"
    )
    results["kd_2"] = kd2_fitted
    results["r_t"] = r_t_fitted

    return results, errors


def fit_per_column(df, kd_2_i, r_t_i, l_s, kd_1):
    """Fit every column of the dataframe"""
    L_T_name = df.columns[0]
    results_list = ["Replicate, KD2 (uM), KD2 std (uM), RT (uM), RT (uM) std\n"]

    # for each condition, fit r_t and kd_2
    for replicate in df:
        if replicate != L_T_name:
            print(f"---- {replicate}")

            results, errors = fit_curve(
                data[L_T_name], data[replicate], kd_2_i, r_t_i, l_s, kd_1
            )

            # get results
            kd_2_fitted = results["kd_2"]
            r_t_fitted = results["r_t"]

            # add results to summary
            perr = np.sqrt(np.diag(errors))
            results_list.append(
                f"{replicate}, {M_to_uM(kd_2_fitted)}, {M_to_uM(perr[0])}, {M_to_uM(r_t_fitted)}, {M_to_uM(perr[1])}\n"
            )

            # plot results
            x_smooth = np.logspace(-8, -3, 100)

            fig, ax = plt.subplots()

            plt.plot(data[L_T_name], data[replicate], "o")
            plt.plot(x_smooth, f_sb_model(x_smooth, kd_2_fitted, r_t_fitted, l_s, kd_1))

            ax.set_xscale("log")
            ax.set_ylabel("Fraction bound")
            ax.set_xlabel("L_T (M)")

            plt.title(
                f"Fitted {replicate}\n"
                f"KD2 = {M_to_uM(kd_2_fitted):.3f} uM\n"
                f"RT = {M_to_uM(r_t_fitted):.3f} uM"
            )
            plt.legend(["data", "fit"])

            # Save
            plt.savefig(Path(output_path, f"column_{replicate}.eps"))

    with open(Path(output_path, f"{df.columns[1]}_per_column.csv"), "w") as f:
        f.writelines(results_list)


def fit_all_columns(df, kd_2_i, r_t_i, l_s, kd_1):
    """Fit all columns of the dataframe together"""
    L_T_name = df.columns[0]
    condition_name = df.columns[1]
    len_df = len(df.columns) - 1

    results_list = ["Condition, KD2 (uM), KD2 std (uM), RT (uM), RT (uM) std\n"]

    # create a new dataframe
    # TODO: there is a better way to do that using pandas...
    L_T = pandas.concat([df[L_T_name] for _ in range(len_df)])
    F_B = pandas.concat([df[c] for c in df if c != L_T_name])

    # fit
    results, errors = fit_curve(L_T, F_B, kd_2_i, r_t_i, l_s, kd_1)

    # print results
    kd_2_fitted = results["kd_2"]
    r_t_fitted = results["r_t"]

    # add results to summary
    perr = np.sqrt(np.diag(errors))
    results_list.append(
        f"{condition_name}, {M_to_uM(kd_2_fitted)}, {M_to_uM(perr[0])}, {M_to_uM(r_t_fitted)}, {M_to_uM(perr[1])}\n"
    )

    # plot results
    x_smooth = np.logspace(-8, -3, 100)

    fig, ax = plt.subplots()

    plt.plot(L_T, F_B, "o")
    plt.plot(x_smooth, f_sb_model(x_smooth, kd_2_fitted, r_t_fitted, l_s, kd_1))

    ax.set_xscale("log")
    ax.set_ylabel("Fraction bound")
    ax.set_xlabel("L_T (M)")

    plt.title(
        f"Fitted {condition_name}\n"
        f"kd2 = {M_to_uM(kd_2_fitted):.3f} uM\n"
        f"rt = {M_to_uM(r_t_fitted):.3f} uM"
    )
    plt.legend(["data", "fit"])

    # Save
    plt.savefig(Path(output_path, f"{condition_name}_all.eps"))

    with open(Path(output_path, f"{condition_name}_all_columns.csv"), "w") as f:
        f.writelines(results_list)


def open_csv(path):
    """Read .csv file"""
    return pandas.read_csv(path, encoding="utf-8-sig")


def process_data(df):
    """Process a data frame by converting the first column from
    uM to M and by dividing all other columns by 100 then removing
    the lowest value of the entire data frane (conditions).
    """
    X = df.columns[0]

    # first, convert X from uM to M
    df[X] = uM_to_M(df[X])

    # convert all the other columns to % between 0 and 1
    min_val = 100
    for column in df:
        if column != X:
            df[column] /= 100

            if df[column].min() < min_val:
                min_val = df[column].min()

    # subtract offset
    for column in df:
        if column != X:
            df[column] -= min_val

    # split into two dataframes
    # TODO: there must be a better way using pandas
    len_data = 1 + (len(df.columns) - 1) // 2
    left = [X]
    right = [X]
    for i, column in enumerate(df):
        if column != X:
            if i < len_data:
                left.append(column)
            else:
                right.append(column)

    df1 = df[left]
    df2 = df[right]

    return df1, df2, X


def plot_paper_figure_3a():
    """Plot figure 3a from the original paper"""
    # Paper example
    x_paper = np.logspace(-8, -3, 100)  # L_T

    # parameters
    r_t_paper = uM_to_M(1.0)
    l_st_paper = uM_to_M(30 / 1_000)
    kd_1_paper = uM_to_M(1.0)
    kd_2_paper = uM_to_M(np.array([0.001, 0.01, 0.1, 1, 10, 100]))  # 0.01 uM to 1000 uM

    fig, ax = plt.subplots()
    for kd2 in kd_2_paper:
        plt.plot(x_paper, f_sb_model(x_paper, kd2, r_t_paper, l_st_paper, kd_1_paper))
    ax.set_xscale("log")
    ax.set_ylabel("Fraction bound")
    ax.set_xlabel("L_t (M)")
    plt.legend(["Kd_2 (uM): %3.2f" % M_to_uM(k_i) for k_i in kd_2_paper], fontsize=8)
    plt.title("Paper Fig 3a")


def generate_example(plot=True, save=False):
    """Generate a noisy example"""
    x_vals = np.logspace(-8, -3, 8)
    x_smooth = np.logspace(-8, -3, 100)

    # parameters
    l_st_paper = uM_to_M(30 / 1_000)
    kd_1_paper = uM_to_M(1.0)

    r_t_example = uM_to_M(1.0)
    kd_2_example = uM_to_M(1.0)

    # compute f_sb
    f_sb = f_sb_model(x_vals, kd_2_example, r_t_example, l_st_paper, kd_1_paper)
    f_sb_2 = f_sb_model(x_vals, 10 * kd_2_example, r_t_example, l_st_paper, kd_1_paper)

    # add noise
    f_sb += np.random.normal(0, 1e-2, f_sb.shape)
    f_sb_2 += np.random.normal(0, 1e-2, f_sb.shape)

    # create replicates
    f_sb_r2 = f_sb_model(
        x_vals, kd_2_example, r_t_example, l_st_paper, kd_1_paper
    ) + np.random.normal(0, 1e-2, f_sb.shape)
    f_sb_2_r2 = f_sb_model(
        x_vals, 10 * kd_2_example, r_t_example, l_st_paper, kd_1_paper
    ) + np.random.normal(0, 1e-2, f_sb.shape)

    # make sure there is no negative value
    f_sb[f_sb < 0] = 0
    f_sb_2[f_sb_2 < 0] = 0

    f_sb_r2[f_sb_r2 < 0] = 0
    f_sb_2_r2[f_sb_2_r2 < 0] = 0

    # initial value
    kd_2_initial = uM_to_M(1.2)
    print(f"Condition 1 kd_2={M_to_uM(kd_2_example)} and r_t={M_to_uM(r_t_example)}")
    print(f"Condition 2 kd_2={M_to_uM(10*kd_2_example)} and r_t={M_to_uM(r_t_example)}")

    # fit fsb 1
    results, _ = fit_curve(
        x_vals, f_sb, kd_2_initial, r_t_example, l_st_paper, kd_1_paper
    )
    kd_2_fitted = results["kd_2"]
    r_t_fitted = results["r_t"]

    # fit the second one
    results, _ = fit_curve(
        x_vals, f_sb_2, kd_2_initial, r_t_example, l_st_paper, kd_1_paper
    )
    kd_2_fitted_2 = results["kd_2"]
    r_t_fitted_2 = results["r_t"]

    # plot
    if plot:
        fig, ax = plt.subplots()

        plt.plot(x_vals, f_sb, "o")
        plt.plot(
            x_smooth,
            f_sb_model(x_smooth, kd_2_fitted, r_t_fitted, l_st_paper, kd_1_paper),
        )

        plt.plot(x_vals, f_sb_2, "o")
        plt.plot(
            x_smooth,
            f_sb_model(x_smooth, kd_2_fitted_2, r_t_fitted_2, l_st_paper, kd_1_paper),
        )

        ax.set_xscale("log")
        ax.set_ylabel("Fraction bound")
        ax.set_xlabel("L_t (M)")
        plt.title("Generated example")
        plt.legend(["Condition 1", "Fit Condition 1", "Condition 2", "Fit Condition 2"])

    # save result
    if save:
        offset = 7
        values = ["L_T (uM), C1 r1, C1 r2, C2 r1, C2 r2\n"]
        for i in range(len(x_vals)):
            values.append(
                f"{'%.2E' % M_to_uM(x_vals[i])}, "
                f"{100 * f_sb[i] + offset :.2f}, "
                f"{100 * f_sb_r2[i] + offset :.2f}, "
                f"{100 * f_sb_2[i] + offset :.2f}, "
                f"{100 * f_sb_2_r2[i] + offset :.2f}\n"
            )

        with open("example.csv", "w") as f:
            f.writelines(values)

## 1 - Parameters

Set the parameters of the notebook here:

- `input_file`: the path to the data
- `output_path`: path to the folder where to save the figures

Then, the parameters of the experiment:

- `l_s_value`: l_s, total concentration of the labeled ligand (probe) in M
- `kd_1_value`: kd_1, dissociation constant of the probe interaction

And finally the parameters of the fitting:

- `r_t_initial`: initial guess of the value of r_t, total receptor concentration in M
- `kd_2_initial`: initial guess of the value of kd_2, dissociation constant of the competitor interaction


**Note**: If you want to use `example.csv`, the experimental parameters are the following:
```python
l_s_value = uM_to_M(30 / 1_000)
kd_1_value = uM_to_M(1.0)
r_t_initial = uM_to_M(1.0)
kd_2_initial = uM_to_M(1.0)
```

In [ ]:
# notebook parameters
input_file = Path("data_competition_assay.csv")
output_path = Path("results/")

In [ ]:
# experimental parameters
l_s_value = uM_to_M(0.3)
kd_1_value = uM_to_M(0.49)

# fitting parameters
r_t_initial = uM_to_M(0.6)
kd_2_initial = uM_to_M(0.5)

## Controls

The next two cells are just controls to see that the model is correct and that the fitting procedure work.

In [ ]:
# Optional: run the figure from the paper to verify that the model is properly implemented
plot_paper_figure_3a()

In [ ]:
# Optional: generate an example of a fit over noisy data
generate_example()

## Load and reshape data

Here we load the data and reshape it:
- `L_T` to M
- Conditions to % between 0 and 1 (no normalization, just conversion by dividing by 100)
- Remove offset (minimum value over all conditions)

In [ ]:
# load data
data = open_csv(input_file)

# process data
condition_1, condition_2, L_T = process_data(data)
print(f"Condition 1:\n{condition_1}")
print(f"Condition 2:\n{condition_2}")

## Fit each individual replicate in each condition

Here the model is fitted on a per column basis. The results are saved in a summary `.csv` file.

In [ ]:
#################
## Condition 1 ##
#################
fit_per_column(condition_1, kd_2_initial, r_t_initial, l_s_value, kd_1_value)

In [ ]:
#################
## Condition 2 ##
#################
fit_per_column(condition_2, kd_2_initial, r_t_initial, l_s_value, kd_1_value)

## Fit all replicates together for each condition

We aggregate all replicates and fit them to a single model, the result is exported to a `.csv`.

In [ ]:
fit_all_columns(condition_1, kd_2_initial, r_t_initial, l_s_value, kd_1_value)

In [ ]:
fit_all_columns(condition_2, kd_2_initial, r_t_initial, l_s_value, kd_1_value)